# Loading

In [ ]:
!pip install -U google-genai

In [90]:
import pandas as pd
import numpy as np
import copy
import string
import re
import json
import random
import time
from tqdm import tqdm

from google import genai
from google.genai import types

In [4]:
client = genai.Client(api_key="")

In [ ]:
for model_info in client.models.list():
    print(model_info.name)

In [6]:
for model_info in client.tunings.list():
    print(model_info.name)

# Dataset Prep

In [7]:
train_df = pd.read_csv('data/seniority_labelled_development_set_cleaned.csv')
test_df = pd.read_csv('data/seniority_labelled_test_set_cleaned.csv')

In [8]:
# df to store model predictions
train_format_df = pd.DataFrame(columns=["text_input", "output"])

In [9]:
test_format_df = pd.DataFrame(columns=["text_input", "output"])

In [10]:
for i in range(len(train_df)):

    desc = f"""job_title: {train_df.iloc[i].job_title},
job_summary: {train_df.iloc[i].job_summary},
job_ad_details: {train_df.iloc[i].job_ad_details},
classification_name: {train_df.iloc[i].classification_name},
subclassification_name: {train_df.iloc[i].subclassification_name},
"""

    desc += """
Extract seniority label from this job description. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, intermediate to senior, senior executive] . If not present in the set, then create a label. "
Respond in JSON: {"seniority_label": ""}.
"""

    label = f'{{"seniority_label": "{train_df.iloc[i].y_true}"}}.'
    train_format_df.loc[i] = [desc, label]

In [11]:
train_format_df

,text_input,output
0,job_title: Electrical BIM modeller - Contract ...,"{""seniority_label"": ""intermediate""}."
1,"job_title: Marketing Manager - Tourism,\njob_s...","{""seniority_label"": ""senior""}."
2,job_title: Quality Coordinator Registered Nurs...,"{""seniority_label"": ""lead""}."
3,"job_title: National Lean and Quality Manager,\...","{""seniority_label"": ""head""}."
4,job_title: Experienced Registered Nurses: casu...,"{""seniority_label"": ""experienced""}."
...,...,...
2747,"job_title: Truck Driver - Pneumatic Tanker,\nj...","{""seniority_label"": ""intermediate""}."
2748,job_title: Senior UI Designer | Front End Deve...,"{""seniority_label"": ""senior""}."
2749,job_title: Virtual Pharmaceutical Sales Repres...,"{""seniority_label"": ""entry-level""}."
2750,"job_title: Electrical Design and Draftsperson,...","{""seniority_label"": ""experienced""}."


In [24]:
train_format_df.to_csv('seniority_dataset.csv')

In [12]:
for i in range(len(test_df)):

    desc = f"""job_title: {test_df.iloc[i].job_title},
job_summary: {test_df.iloc[i].job_summary},
job_ad_details: {test_df.iloc[i].job_ad_details},
classification_name: {test_df.iloc[i].classification_name},
subclassification_name: {test_df.iloc[i].subclassification_name},
"""

    desc += """
Extract seniority label from this job description. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, intermediate to senior, senior executive] . If not present in the set, then create a label. "
Respond in JSON: {"seniority_label": ""}.
"""

    label = test_df.iloc[i].y_true
    test_format_df.loc[i] = [desc, label]

In [13]:
test_format_df

,text_input,output
0,job_title: Financial Planning and Modelling Ma...,senior
1,job_title: Residential Site Manager - Melbourn...,experienced
2,job_title: Wellbeing Administrator - Fixed Ter...,entry-level
3,job_title: Senior Natural Hazards and Climate ...,senior
4,job_title: Credit Union - Member & Customer se...,intermediate
...,...,...
684,"job_title: GP Family Planning,\njob_summary: D...",experienced
685,"job_title: Dispatch Team Member - Kerikeri,\nj...",entry-level
686,job_title: Senior Property Development Officer...,senior
687,"job_title: Client Services Associate - Mosman,...",associate


# Tuning

In [14]:
# create tuning examples
training_dataset=types.TuningDataset(
        examples=[
            types.TuningExample(
                text_input=train_format_df.iloc[i].text_input,
                output=train_format_df.iloc[i].output,
            )
            for i in range(len(train_format_df))
        ],
    )

In [15]:
# start tuning job
tuning_job = client.tunings.tune(
    base_model='models/gemini-1.5-flash-001-tuning',
    training_dataset=training_dataset,
    # hyperparameters
    config=types.CreateTuningJobConfig(
        epoch_count=5,
        batch_size=4,
        learning_rate=0.001,
        tuned_model_display_name="seniority_tuned_model_5"
    )
)

/var/folders/7f/xkmyr8ss5zzgc7z2m_hqb_f80000gn/T/ipykernel_2185/1446187548.py:2: ExperimentalWarning: The SDK's tuning implementation is experimental, and may change in future versions.
  tuning_job = client.tunings.tune(


In [99]:
tuning_job

TuningJob(name='tunedModels/senioritytunedmodel5-addujvt0uujq', state=<JobState.JOB_STATE_SUCCEEDED: 'JOB_STATE_SUCCEEDED'>, create_time=datetime.datetime(2025, 4, 24, 9, 17, 40, 115709, tzinfo=TzInfo(UTC)), start_time=datetime.datetime(2025, 4, 24, 9, 17, 40, 561293, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2025, 4, 24, 11, 26, 41, 818239, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 4, 24, 11, 26, 41, 818239, tzinfo=TzInfo(UTC)), error=None, description='{"description":"","exampleInput":"","exampleOutput":"","showedTuningComplete":false,"rowsCount":2752}', base_model='models/gemini-1.5-flash-001-tuning', tuned_model=TunedModel(model='tunedModels/senioritytunedmodel5-addujvt0uujq', endpoint='tunedModels/senioritytunedmodel5-addujvt0uujq'), supervised_tuning_spec=None, tuning_data_stats=None, encryption_spec=None, partner_model_tuning_spec=None, distillation_spec=None, experiment=None, labels=None, pipeline_job=None, tuned_model_display_name=None)

# Testing

In [100]:
# fetch the tuning job again by model name

tuning_job = client.tunings.get(name='tunedModels/senioritytunedmodel5-addujvt0uujq')
print(tuning_job)

name='tunedModels/senioritytunedmodel5-addujvt0uujq' state=<JobState.JOB_STATE_SUCCEEDED: 'JOB_STATE_SUCCEEDED'> create_time=datetime.datetime(2025, 4, 24, 9, 17, 40, 115709, tzinfo=TzInfo(UTC)) start_time=datetime.datetime(2025, 4, 24, 9, 17, 40, 561293, tzinfo=TzInfo(UTC)) end_time=datetime.datetime(2025, 4, 24, 11, 26, 41, 818239, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 4, 24, 11, 26, 41, 818239, tzinfo=TzInfo(UTC)) error=None description='{"description":"","exampleInput":"","exampleOutput":"","showedTuningComplete":false,"rowsCount":2752}' base_model='models/gemini-1.5-flash-001-tuning' tuned_model=TunedModel(model='tunedModels/senioritytunedmodel5-addujvt0uujq', endpoint='tunedModels/senioritytunedmodel5-addujvt0uujq') supervised_tuning_spec=None tuning_data_stats=None encryption_spec=None partner_model_tuning_spec=None distillation_spec=None experiment=None labels=None pipeline_job=None tuned_model_display_name=None


## Qualitative

In [105]:
test_index = 174

response = client.models.generate_content(
    # change model name
    model='tunedModels/senioritytunedmodel5-addujvt0uujq',
    contents=test_format_df.iloc[test_index].text_input,
)

response.text

'entry-level"}.'

In [104]:
test_index = 2

response = client.models.generate_content(
    # change model name
    model='tunedModels/senioritytunedmodel5-addujvt0uujq',
    contents=test_format_df.iloc[test_index].text_input,
)

response.text

'{"seniority_label": "entry-level"}.'

## Quantitative

In [108]:
# create df to capture results
results_df = pd.DataFrame(columns=["y_true", "y_pred"])

In [109]:
for i in (pbar := tqdm(range(len(test_format_df)))):
    pbar.set_description("")
    while True:
        try:
            response = client.models.generate_content(
                model='tunedModels/senioritytunedmodel5-addujvt0uujq',
                contents=test_format_df.iloc[i].text_input,
            )
            results_df.loc[len(results_df)] = [test_format_df.iloc[i].output, response.text]
            break  # break the retry loop if successful

        except Exception as e:
            retry_delay = 60  # default delay
            
            pbar.set_description(f"Waiting {retry_delay}s to retry at index {i}")
            time.sleep(retry_delay)

100%|█████████████████████████████████████████| 689/689 [12:41<00:00,  1.11s/it]


## Post-process predcitions

In [121]:
results_df[results_df.y_pred.isna()]

,y_true,y_pred
6,experienced,None
56,senior,None
160,intermediate,None
173,experienced,None
267,intermediate,None
373,entry-level,None
409,entry-level,None
536,junior,None
541,intermediate,None
588,entry-level,None


In [152]:
def process_pred(y_pred):
    if y_pred == None:
        return ""
    y_pred = y_pred.replace('“', '"')
    y_pred = y_pred.replace('”', '"')
        
    if '{' in y_pred and ':' in y_pred:
        y_pred_ = y_pred[y_pred.find('{'):y_pred.find('}') + 1]
        
        try:
            answer = json.loads(y_pred_)
            label = answer['seniority_label'] if 'seniority_label' in answer else answer['seniority-label']
        except Exception as e:
            print(f"{e}: {y_pred_}")
            label = "ERROR " + y_pred
    else:
        y_pred_ = y_pred.translate(str.maketrans('', '', '{".' + "'"))
        label = y_pred_[:y_pred_.find('}')]
        
    return label

In [153]:
results_df['y_pred_processed'] = results_df.y_pred.map(process_pred)

In [154]:
results_df

,y_true,y_pred,y_pred_processed
0,senior,"head""}.",head
1,experienced,"experienced""}.",experienced
2,entry-level,"{""seniority_label"": ""intermediate""}.",intermediate
3,senior,"senior""}.",senior
4,intermediate,"entry-level""}.",entry-level
...,...,...,...
684,experienced,"{""seniority_label"": ""intermediate""}.",intermediate
685,entry-level,"entry-level""}.",entry-level
686,senior,"senior""}.",senior
687,associate,"{""seniority_label"": ""intermediate""}.",intermediate


In [155]:
for i in range(results_df.shape[0]):
    y_pred_processed = results_df.iloc[i].y_pred_processed
    
    for c in y_pred_processed:
        if not c.isalpha():
            print(y_pred_processed)
            break

entry-level
entry-level
entry-level
entry-apprentice
entry-level
entry-level
entry-level
senior assistant
entry-level
lead head
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
senior assistant
junior-mid-senior
entry-level
senior assistant
entry-level
entry-level
entry-level
entry-level
entry-seniority
junior supervisor
intermediate-junior
entry-level
head-head
entry-level
entry-level
entry-level
mid-senior
seniority-assistant
head headhead
entry-level
entry-level
entry-level
entry-level
entry-level
experienced head
entry-level
seniority_label: experienced
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-qualified
entry-level
entry-level
seniority_label: assistant
entry-level
entry-level
entry-level
entry-level
mid-senior
entry-level
head senior
entry-level
entry-level
entry-level
entry-level

In [157]:
def final_process(y_pred_processed):
    if 'seniority_label: ' in y_pred_processed:
        y_pred_processed = y_pred_processed[len('seniority_label: '):]
    return y_pred_processed

In [158]:
results_df['y_pred_processed'] = results_df.y_pred_processed.map(final_process)

In [159]:
for i in range(results_df.shape[0]):
    y_pred_processed = results_df.iloc[i].y_pred_processed
    
    for c in y_pred_processed:
        if not c.isalpha():
            print(y_pred_processed)
            break

entry-level
entry-level
entry-level
entry-apprentice
entry-level
entry-level
entry-level
senior assistant
entry-level
lead head
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
senior assistant
junior-mid-senior
entry-level
senior assistant
entry-level
entry-level
entry-level
entry-level
entry-seniority
junior supervisor
intermediate-junior
entry-level
head-head
entry-level
entry-level
entry-level
mid-senior
seniority-assistant
head headhead
entry-level
entry-level
entry-level
entry-level
entry-level
experienced head
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
entry-qualified
entry-level
entry-level
entry-level
entry-level
entry-level
entry-level
mid-senior
entry-level
head senior
entry-level
entry-level
entry-level
entry-level
entry-senior
entry-level
entry-level
entry-level
entry-

In [160]:
# export the dataframe to a new csv file
results_df.to_csv('seniority_labelled_test_set_gemini_finetune.csv', index=False)

# Metrics

In [1]:
import json
import string
import pandas as pd
import numpy as np
import ast

In [2]:
def categories(label):
    lab = str(label).lower()
    if 'entry' in lab:
        return 'Entry'
    elif 'junior' in lab or 'assistant' in lab:
        return 'Junior'
    elif 'intermediate' in lab or 'experienced' in lab or 'mid' in lab:
        return 'Mid'
    elif 'senior' in lab or 'lead' in lab:
        return 'Senior'
    elif any(x in lab for x in ['manager','director','chief','head','executive','principal']):
        return 'Leadership'
    else:
        return 'Other'
    
def process(row):
    if "error" not in row:
        row = row.strip().lower()
        if '[' in row and ']' in row:
            row = ast.literal_eval(row[row.find('['):row.find(']')+1])
            row = '/'.join(row)
        return row
    else:
        row = row.strip().lower()
        try:
            row = row[row.find('{'):row.find('}')+1]
            row_data = json.loads(row)
        except Exception:
            row = row[len('error'):]
            row = row.translate(str.maketrans('', '', string.punctuation))
            row = row.strip()
            if len(row.split(' ')) == 1:
                return row
            return row
        row_data = row_data['clue'] #if 'clue' in row_data else 
        if row_data == 'yes':
            return ""
        return row_data

def get_accuracy(path_to_preds):
    preds = pd.read_csv(path_to_preds)
    test_df = pd.read_csv('data/seniority_labelled_test_set_cleaned.csv')
    
    test_df['y_pred'] = preds.values.reshape(-1)
    test_df = test_df.fillna('')
    
    test_df['y_pred'] = test_df['y_pred'].map(process)
    test_df['y_true'] = test_df['y_true'].map(process)
    
    test_df['y_pred_cat'] = test_df['y_pred'].map(categories)
    test_df['y_true_cat'] = test_df['y_true'].map(categories)
    
    exact = (test_df['y_pred'] == test_df['y_true']).mean() * 100
    cat = ((test_df['y_pred'] != test_df['y_true']) & (test_df['y_pred_cat'] == test_df['y_true_cat'])).mean() * 100
    overall = ((test_df['y_pred'] == test_df['y_true']) | (test_df['y_pred_cat'] == test_df['y_true_cat'])).mean() * 100
    
    exact_count = (test_df['y_pred'] == test_df['y_true']).sum()
    cat_count = ((test_df['y_pred'] != test_df['y_true']) & (test_df['y_pred_cat'] == test_df['y_true_cat'])).sum()
    overall_count = ((test_df['y_pred'] == test_df['y_true']) | (test_df['y_pred_cat'] == test_df['y_true_cat'])).sum()
    
    print(f'Exact: {exact_count}/{test_df.shape[0]}')
    print(f'Similar: {cat_count}/{test_df.shape[0]}')
    print(f'Overall: {overall_count}/{test_df.shape[0]}')
    
    res = pd.DataFrame(
        {
            'Exact': round(exact, 2),
            'Similar': round(cat, 2),
            'Overall': round(overall, 2),
        },
        index=['Accuracy (%)']
    )
    
    return res

In [4]:
path_to_preds = 'seniority_labelled_test_set_gemini_finetune.csv'
test_df = pd.read_csv(path_to_preds)
test_df['y_pred'] = test_df.y_pred_processed
test_df = test_df.fillna('')

In [5]:
test_df['y_pred'] = test_df['y_pred'].map(process)
test_df['y_true'] = test_df['y_true'].map(process)

test_df['y_pred_cat'] = test_df['y_pred'].map(categories)
test_df['y_true_cat'] = test_df['y_true'].map(categories)

exact = (test_df['y_pred'] == test_df['y_true']).mean() * 100
cat = ((test_df['y_pred'] != test_df['y_true']) & (test_df['y_pred_cat'] == test_df['y_true_cat'])).mean() * 100
overall = ((test_df['y_pred'] == test_df['y_true']) | (test_df['y_pred_cat'] == test_df['y_true_cat'])).mean() * 100

exact_count = (test_df['y_pred'] == test_df['y_true']).sum()
cat_count = ((test_df['y_pred'] != test_df['y_true']) & (test_df['y_pred_cat'] == test_df['y_true_cat'])).sum()
overall_count = ((test_df['y_pred'] == test_df['y_true']) | (test_df['y_pred_cat'] == test_df['y_true_cat'])).sum()

print(f'Exact: {exact_count}/{test_df.shape[0]}')
print(f'Similar: {cat_count}/{test_df.shape[0]}')
print(f'Overall: {overall_count}/{test_df.shape[0]}')

pd.DataFrame(
    {
        'Exact': round(exact, 2),
        'Similar': round(cat, 2),
        'Overall': round(overall, 2),
    },
    index=['Accuracy (%)']
)

Exact: 362/689
Similar: 103/689
Overall: 465/689


,Exact,Similar,Overall
Accuracy (%),52.54,14.95,67.49
